In [2]:
import os

In [3]:
%pwd

'c:\\Users\\user\\Desktop\\BHS\\Coding\\3_ML_Ops_Pipeline_AWS\\research'

In [4]:
os.chdir("c:\\Users\\user\\Desktop\\BHS\\Coding\\3_ML_Ops_Pipeline_AWS")

In [5]:
%pwd

'c:\\Users\\user\\Desktop\\BHS\\Coding\\3_ML_Ops_Pipeline_AWS'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="http://ec2-18-184-239-229.eu-central-1.compute.amazonaws.com:5000/"
#os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
#os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a35333f9eac5b10041d5c8edbcef0"

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="http://ec2-3-68-192-238.eu-central-1.compute.amazonaws.com:5000/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [16]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-07 20:30:13,140: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-07 20:30:13,144: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-07 20:30:13,146: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 19s 3s/step - loss: 11.6552 - accuracy: 0.5686
[2024-10-07 20:30:33,252: INFO: common: json file saved at: scores.json]


2024/10/07 20:30:34 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-10-07 20:30:35,158: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpq_hncku2\model\data\model\assets
[2024-10-07 20:30:35,798: INFO: builder_impl: Assets written to: C:\Users\user\AppData\Local\Temp\tmpq_hncku2\model\data\model\assets]
[2024-10-07 20:30:45,437: INFO: credentials: Found credentials in shared credentials file: ~/.aws/credentials]


S3UploadFailedError: Failed to upload C:\Users\user\AppData\Local\Temp\tmpq_hncku2\model\conda.yaml to mlflobucket12/0/bcafd9826566444bba0848150e537b09/artifacts/model/conda.yaml: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist